# Lesson 4: 학습을 위한 모델 준비

Deeplearning.AI & Upstage의 다음 강의를 듣고 정리한 노트북입니다.

https://learn.deeplearning.ai/courses/pretraining-llms/lesson/5/model-initialization

이 노트북에서는 pretrained LLM 훈련을 위해 모델을 초기화하는 여러 방법에 대해 학습합니다.

__** M1 Mac은 `torch.bfloat16` 학습 미지원 **__

In [1]:
# Ignore insignificant warnings (ex: deprecation warnings)
import warnings
warnings.filterwarnings('ignore')

# Set a seed value for reproducibility
import torch

def fix_torch_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

fix_torch_seed()

## 모델 설정

Meta의 Llama 모델에 기반하여 모델을 설정하고자 합니다. 먼저 Llama모델의 설정을 불러온 뒤, 우리가 원하는 대로 일부 파라미터를 수정합니다.

In [2]:
from transformers import LlamaConfig
config = LlamaConfig()
print(config)

LlamaConfig {
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 2048,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "transformers_version": "4.37.2",
  "use_cache": true,
  "vocab_size": 32000
}



In [3]:
config.num_hidden_layers = 12      # reduced from 32 to 12
config.hidden_size = 1024          # reduced 1/4 from 4096 to 1024
config.intermediate_size = 4096    # reduced 1/3 from 11008 to 4096 (dimension of MLP representations)
config.num_key_value_heads = 8     # reduced 1/4 from 32 to 8 (defaults to num_attention_heads=32)
config.torch_dtype = "bfloat16"    # for half-precision training
config.use_cache = False           # `True` is incompatible w/ gradient checkpointing
print(config)

LlamaConfig {
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "max_position_embeddings": 2048,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 12,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.37.2",
  "use_cache": false,
  "vocab_size": 32000
}



## 모델 초기화

여기서는 다음 4가지 모델 초기화 방법을 학습합니다.

1. 랜덤 초기화(Random weight initialization)
2. 기존 모델 웨이트 사용(Using an existing model for continued pre-training)
3. 기존 모델 다운스케일링(Downscaling an exisiting model)
4. 기존 모델 업스케일링(Upscaling an existing model)

### 랜덤 초기화(Random weight initialization)

모델 웨이트를 랜덤하게 초기화합니다. 이때 값들은 평균이 0이고 표준편차가 0.02인 정규분포를 따르며, $|x| \ge 0.04$ 인 값들은 0으로 변환됩니다(truncated normal distribution).

In [4]:
from transformers import LlamaForCausalLM
model = LlamaForCausalLM(config)
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 1024)
    (layers): ModuleList(
      (0-11): 12 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (k_proj): Linear(in_features=1024, out_features=256, bias=False)
          (v_proj): Linear(in_features=1024, out_features=256, bias=False)
          (o_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=1024, out_features=4096, bias=False)
          (up_proj): Linear(in_features=1024, out_features=4096, bias=False)
          (down_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Line

In [5]:
def print_nparams(model):
    """Calculate the total number of model parameters"""
    nparams = sum(p.numel() for p in model.parameters())
    print(f"The total number of parameters is: {nparams}")

print_nparams(model)  # 248013824 => 248M

The total number of parameters is: 248013824


랜덤하게 초기화된 값들을 조회해봅니다.

In [11]:
layer_name = "model.layers.0.self_attn.q_proj.weight"

for name, param in model.named_parameters():
    if name == layer_name:
        print(f"First 30 weights of layer '{layer_name}':")
        print(param.data.view(-1)[:30])
        break

First 30 weights of layer 'model.layers.0.self_attn.q_proj.weight':
tensor([ 1.5794e-02, -2.2748e-02,  2.0156e-02, -2.6072e-02, -8.3267e-05,
         8.7432e-03, -9.0255e-04, -4.2442e-02,  1.5337e-02,  1.4482e-02,
         1.3526e-02,  1.9171e-03, -2.3141e-02, -4.2336e-03,  6.9818e-04,
         8.9955e-03, -2.0524e-02, -1.3378e-02,  2.3255e-02,  9.5166e-04,
         2.1053e-02,  1.2794e-02, -7.6783e-03, -3.7832e-03, -8.9180e-03,
         7.4018e-04, -2.5204e-02, -1.7069e-02,  1.3481e-03,  4.7622e-02])


이 상태에서 inference를 진행하면 어떻게 될까요? 당연히 알 수 없는 말들을 생성해냅니다.

In [15]:
from transformers import LlamaTokenizer, TextStreamer
model_name_or_path = 'upstage/SOLAR-10.7B-v1.0'
tokenizer = LlamaTokenizer.from_pretrained(model_name_or_path)

prompt = "I am an engineer. I love"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
streamer = TextStreamer(
    tokenizer,
    skip_prompt=True,
    skip_special_tokens=True,
)

outputs = model.generate(
    **inputs,
    streamer=streamer,
    use_cache=True,
    max_new_tokens=128,
    do_sample=False
)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


possessed possessed possessed possessed possessed possessedcontinuecontinuecontinuecontinuecontinueDownloadџcontinueDownloadcontinueDownloadcontinueertsxE Point remoterts remoterts remoterts갑continuecontinuecontinue wide wide atr wide atr wide wide wide wide wide wide wide wide wide wide wide wideursor otra FC otraopesopesopesopesopesopesopesopesopesopesopes wideopes wideopes wideopes wideopes wideopes wideopes wideopesimpse Library wideopesasterasterasterasterasterasterasterasterasterasterasterasterasterasterasterasterasterasterasteraster primarily primarily primarily primarily primarily primarily primarilyasterasterasterasterasterasterasterasterasterasterasteraster primarilyitä primarilyitä primarilyitä primarilyitä


메모리 이슈를 방지하고자 모델을 메모리에서 삭제합니다.

In [16]:
import gc
del model
del streamer
del outputs
gc.collect()

523

### 기존 모델 웨이트 사용(Reuse general pretrained model weights)

이미 학습된 기존 모델의 웨이트를 가져와서, 여기서부터 우리의 데이터로 학습을 시작할 수 있습니다.

In [17]:
from transformers import AutoModelForCausalLM

model_name_or_path = "upstage/TinySolar-248m-4k"
model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    device_map="cpu",
    torch_dtype=torch.bfloat16,
)

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [19]:
print_nparams(model)

The total number of parameters is: 248013824


### 기존 모델 다운스케일링(Downscaling from a general pretrained model)

다운스케일링은 이미 학습된 모델의 레이어 중 일부만을 사용하여 모델의 크기를 축소시키는 방법입니다. 일반적으로 레이어 계층의 중간 부분을 삭제하며, 기존 모델이 큰 경우에 유용한 방식입니다.

In [20]:
from transformers import AutoTokenizer, AutoConfig

model_name_or_path = "upstage/TinySolar-248m-4k"
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

tokenizer_config.json:   0%|          | 0.00/966 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [30]:
layers = model.model.layers
len(layers)

12

기존 모델은 레이어가 12개로 이루어져있는데, 이 중 5, 6번 레이어를 삭제합니다.

In [31]:
model.model.layers = layers[:5] + layers[-5:]

config = AutoConfig.from_pretrained(
    model_name_or_path,
    num_hidden_layers=len(model.model.layers)
)
model.config = config

print_nparams(model)

The total number of parameters is: 217601024


파라미터의 갯수가 248M에서 217M으로 감소하였습니다.

In [32]:
del model
gc.collect()

1004

### 기존 모델 업스케일링(Depth Upscaling from a general pretrained model)

업스케일링은 기존 모델의 레이어를 이용하여 레이어의 갯수를 늘리는 방법입니다. 여기서는 `tinySolar-248m-4k` 모델을 이용하여, 레이어를 12개에서 16개로 확대합니다. 이를 위해 다음과 같이 진행합니다.

1. 레이어가 16개인 모델을 랜덤하게 초기화
2. 레이어가 12개인 `tinySolar-248m-4k` 모델 로드
3. `tinySolar-248m-4k` 모델의 레이어 중 1~8번, 5~12번 레이어를 각각 랜덤하게 초기화한 모델의 1~8번, 9~16번 레이어에 복사
4. `tinySolar-248m-4k` 모델의 임베딩/분류 레이어를 랜덤 초기화 모델에 복사

#### 레이어가 16개인 모델 랜덤 초기화

In [47]:
config = LlamaConfig(
    num_hidden_layers=16, # 레이어가 16개
    hidden_size=1024,
    intermediate_size=4096,
    num_attention_heads=32,
    num_key_value_heads=8,
    # torch_dtype="bfloat16", # M1 mac상에서 학습 미지원
    use_cache=False,
)

print(config)

LlamaConfig {
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "max_position_embeddings": 2048,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 16,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "transformers_version": "4.37.2",
  "use_cache": false,
  "vocab_size": 32000
}



In [48]:
model = LlamaForCausalLM(config)
# model = model.to(dtype=torch.bfloat16)
print_nparams(model)

The total number of parameters is: 308839424


#### 기존 학습된 모델 로드

In [49]:
model_name_or_path = "upstage/TinySolar-248m-4k"
pretrained_model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    device_map="cpu",
    # torch_dtype=torch.bfloat16,
)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

print_nparams(pretrained_model)

The total number of parameters is: 248013824


In [50]:
len(pretrained_model.model.layers)

12

#### 기존 모델로부터 레이어들 복사

In [51]:
from copy import deepcopy

model.model.layers = deepcopy(pretrained_model.model.layers[:-4]) + deepcopy(pretrained_model.model.layers[4:])

model.model.embed_tokens = deepcopy(pretrained_model.model.embed_tokens)
model.lm_head = deepcopy(pretrained_model.lm_head)

print(model.config)

LlamaConfig {
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "max_position_embeddings": 2048,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 16,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "transformers_version": "4.37.2",
  "use_cache": false,
  "vocab_size": 32000
}



In [52]:
print_nparams(model)

The total number of parameters is: 308839424


이렇게 초기화환 모델을 이용하여 inference를 수행합니다. 이제는 (문맥은 이상하지만) 그럴싸한 영어 문장을 생성하는 것을 확인할 수 있습니다.

In [53]:
prompt = "I am an engineer. I love"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

streamer = TextStreamer(
    tokenizer,
    skip_prompt=True,
    skip_special_tokens=True,
)

outputs = model.generate(
    **inputs,
    streamer=streamer,
    use_cache=True,
    max_new_tokens=128,
    do_sample=False
)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


to work with people, but I'm not a good enough teacher. I've been a teacher for 10 years and I've been a teacher for 10 years. I've been a teacher for 10 years and I've been a teacher for 10 years. I've been a teacher for 10 years and I've been a teacher for 10 years.
I've been a teacher for 10 years and I've been a teacher for 10 years. I've been a teacher for 10 years and I've been


## 모델 저장

In [54]:
model.save_pretrained('data/TinySolar-308m-4k-init-nonbf16')